In [1]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

2023-12-19 15:27:36.106301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 15:27:36.262548: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 15:27:37.138343: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.3/lib64
2023-12-19 15:27:37.138432: W te

In [9]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
from tqdm import tqdm
from SmilesPE import tokenizer
def encode(smiles_list,labels,Random=True):
    mols = []
    res_labels = []
    len_list = []
    if Random == True:
        for smiles in smiles_list:
            mol = MolFromSmiles(smiles)
            mols.append(mol)
        res = []
        for index1,mol in (enumerate(tqdm(mols))):
            index = 0
            res_temp = []
            while(len(res_temp)<20 and index <100):
                random_smiles = MolToSmiles(mol, doRandom=True) # e.g 'C(C(=O)O)N' 
                res_temp.append(random_smiles)
                res_temp = list(set(res_temp))
                index+=1
            res += res_temp
            res_labels += [labels[index1]]*len(res_temp)
            len_list.append(len(res_temp))
    else:
        res = smiles_list
        res_labels = labels
        len_list = [1]*len(res_labels)
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        smile_tokens = tokenizer.atomwise_tokenizer(smiles)
        res_train.append(smile_tokens)

    return res_train,res_labels,len_list

In [10]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [5]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [11]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [14]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label,False)
    temp_train.append(res)
    with open('./Tox_data/SMILE_Tox_data/'+tox+'_can','wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:00<00:00, 83593.49it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7385/7385 [00:00<00:00, 155521.65it/s]
Found local copy...
Loading...
Done!
100%|██████████| 655/655 [00:00<00:00, 99897.06it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7278/7278 [00:00<00:00, 140524.53it/s]
Found local copy...
Loading...
Done!
100%|██████████| 475/475 [00:00<00:00, 112667.22it/s]
Found local copy...
Loading...
Done!
100%|██████████| 404/404 [00:00<00:00, 153098.92it/s]
Found local copy...
Loading...
Done!
100%|██████████| 280/280 [00:00<00:00, 88507.43it/s]
Found local copy...
Loading...
Done!
100%|██████████| 1478/1478 [00:00<00:00, 63742.17it/s]


In [18]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label,False))
        with open('./Tox_data/SMILE_Tox_data/'+tox+'_'+label+'_can','wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:03<00:00, 85343.30it/s] 
Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:02<00:00, 112148.06it/s]
Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:03<00:00, 98101.36it/s] 
Found local copy...
Loading...
Done!
100%|██████████| 7265/7265 [00:00<00:00, 156802.66it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6758/6758 [00:00<00:00, 166782.03it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6549/6549 [00:00<00:00, 157448.68it/s]
Found local copy...
Loading...
Done!
100%|██████████| 5821/5821 [00:00<00:00, 161851.95it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6193/6193 [00:00<00:00, 163540.65it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6955/6955 [00:00<00:00, 164210.76it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6450/6450 [00:00<00:00, 171780.91it/s]
Found local copy...
Loading...
Done!
100%|██████████| 5832